# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [42]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot

## Multiple regression

Let's load up the NSFG data again.

In [43]:
import first

live, firsts, others = first.MakeFrames()

Here's birth weight as a function of mother's age (which we saw in the previous chapter).

In [44]:
import statsmodels.formula.api as smf

formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Sun, 19 Mar 2017   Prob (F-statistic):           5.72e-11
Time:                        15:21:38   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000         6.697     6.964
agepreg        0.0175      0.003      6.559      0.000         0.012     0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can extract the parameters.

In [45]:
inter = results.params['Intercept']
slope = results.params['agepreg']
inter, slope

(6.8303969733110481, 0.017453851471802843)

And the p-value of the slope estimate.

In [46]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.7229471073134105e-11

And the coefficient of determination.

In [47]:
results.rsquared

0.0047381154747102583

The difference in birth weight between first babies and others.

In [48]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

The difference in age between mothers of first babies and others.

In [49]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

The age difference plausibly explains about half of the difference in weight.

In [50]:
slope * diff_age

-0.062597099721694707

Running a single regression with a categorical variable, `isfirst`:

In [51]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Sun, 19 Mar 2017   Prob (F-statistic):           2.55e-05
Time:                        15:21:38   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    356.007      0.000         7.286     7.366
isfirst[T.True]    -0.1248      0.030     -4.212      0.000        -0.183    -0.067
==============================================================================
Omnibus:                      988.919   Durbin-Watson:                   1.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2897.107
Skew:                          -0.589   Prob(JB):                         0.00
Kurtosis:                       5.511   Cond. No.                         2.58
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now finally running a multiple regression:

In [52]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Sun, 19 Mar 2017   Prob (F-statistic):           3.95e-11
Time:                        15:21:38   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000         6.762     7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025        -0.131    -0.009
agepreg             0.0154      0.003      5.499      0.000         0.010     0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As expected, when we control for mother's age, the apparent difference due to `isfirst` is cut in half.

If we add age squared, we can control for a quadratic relationship between age and weight.

In [53]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Sun, 19 Mar 2017   Prob (F-statistic):           1.35e-14
Time:                        15:21:38   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000         5.133     6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109        -0.112     0.011
agepreg             0.1124      0.022      5.113      0.000         0.069     0.155
agepreg2           -0.0018      0.000     -4.447      0.000        -0.003    -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

When we do that, the apparent effect of `isfirst` gets even smaller, and is no longer statistically significant.

These results suggest that the apparent difference in weight between first babies and others might be explained by difference in mothers' ages, at least in part.

## Data Mining

We can use `join` to combine variables from the preganancy and respondent tables.

In [54]:
import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

And we can search for variables with explanatory power.

Because we don't clean most of the variables, we are probably missing some good ones.

In [55]:
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'totalwgt_lb ~ agepreg + ' + name
            formula = formula.encode('ascii')

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue
        except patsy.PatsyError:
            raise ValueError(MESSAGE)

        variables.append((results.rsquared, name))

    return variables

In [56]:
variables = GoMining(join)

The following functions report the variables with the highest values of $R^2$.

In [57]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except KeyError:
            print(name, r2)

Some of the variables that do well are not useful for prediction because they are not known ahead of time.

In [58]:
MiningReport(variables)

totalwgt_lb 1.0
birthwgt_lb 0.949812730598 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.300824078447 LOW BIRTHWEIGHT - BABY 1
prglngth 0.130125194886 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.123400413634 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.102031499282 AGE AT TIME OF CONCEPTION
mosgest 0.0271442746396 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.0185509252939 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.0161995035863 RACE
race 0.0161995035863 RACE
nbrnaliv 0.0160177527098 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.0140037955781 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.0134300664657 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.0131024576157 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.0125290225418 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.0121936884045 DURATION OF BREASTFEEDING IN WEEKS

Combining the variables that seem to have the most explanatory power.

In [59]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Sun, 19 Mar 2017   Prob (F-statistic):          4.86e-113
Time:                        15:22:08   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept                6.6303      0.065    102.223      0.000         6.503     6.757
C(race)[T.2]             0.3570      0.032     11.215      0.000         0.295     0.419
C(race)[T.3]             0.2665      0.051      5.175      0.000         0.166     0.367
babysex == 1[T.True]     0.2952      0.026     11.216      0.000         0.244     0.347
nbrnaliv > 1[T.True]    -1.3783      0.108    -12.771      0.000        -1.590    -1.167
paydu == 1[T.True]       0.1196      0.031      3.861      0.000         0.059     0.180
agepreg                  0.0074      0.003      2.921      0.004         0.002     0.012
totincr                  0.0122      0.004      3.110      0.002         0.005     0.020
==============================================================================
Omnibus:                      398.813   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.362
Skew:                          -0.037   Prob(JB):                    3.32e-302
Kurtosis:                       4.947   Cond. No.                         221.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Logistic regression

Example: suppose we are trying to predict `y` using explanatory variables `x1` and `x2`.

In [60]:
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

According to the logit model the log odds for the $i$th element of $y$ is

$\log o = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $

So let's start with an arbitrary guess about the elements of $\beta$:



In [61]:
beta = [-1.5, 2.8, 1.1]

Plugging in the model, we get log odds.

In [62]:
log_o = beta[0] + beta[1] * x1 + beta[2] * x2
log_o

array([-1.5, -0.4, -0.4,  2.4])

Which we can convert to odds.

In [63]:
o = np.exp(log_o)
o

array([  0.22313016,   0.67032005,   0.67032005,  11.02317638])

And then convert to probabilities.

In [64]:
p = o / (o+1)
p

array([ 0.18242552,  0.40131234,  0.40131234,  0.9168273 ])

The likelihoods of the actual outcomes are $p$ where $y$ is 1 and $1-p$ where $y$ is 0. 

In [65]:
likes = np.where(y, p, 1-p)
likes

array([ 0.81757448,  0.40131234,  0.59868766,  0.9168273 ])

The likelihood of $y$ given $\beta$ is the product of `likes`:

In [66]:
like = np.prod(likes)
like

0.18009335296730339

Logistic regression works by searching for the values in $\beta$ that maximize `like`.

Here's an example using variables in the NSFG respondent file to predict whether a baby will be a boy or a girl.

In [67]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live['boy'] = (live.babysex==1).astype(int)

The mother's age seems to have a small effect.

In [68]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Sun, 19 Mar 2017   Pseudo R-squ.:               6.144e-06
Time:                        15:22:10   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
                                        LLR p-value:                    0.7833
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953        -0.185     0.197
agepreg        0.0010      0.004      0.275      0.783        -0.006     0.009
==============================================================================
"""

Here are the variables that seemed most promising.

In [69]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Sun, 19 Mar 2017   Pseudo R-squ.:               0.0001440
Time:                        15:22:10   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
                                        LLR p-value:                    0.8822
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772        -0.234     0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660        -0.122     0.077
C(race)[T.3]    -0.0005      0.083     -0.005      0.996        -0.163     0.162
agepreg         -0.0027      0.006     -0.484      0.629        -0.014     0.008
hpagelb          0.0047      0.004      1.112      0.266        -0.004     0.013
birthord         0.0050      0.022      0.227      0.821        -0.038     0.048
================================================================================
"""

To make a prediction, we have to extract the exogenous and endogenous variables.

In [70]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

The baseline prediction strategy is to guess "boy".  In that case, we're right almost 51% of the time.

In [71]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.507173764518333

If we use the previous model, we can compute the number of predictions we get right.

In [72]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3944.0, 548.0)

And the accuracy, which is slightly higher than the baseline.

In [73]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.51150079708494645

To make a prediction for an individual, we have to get their information into a `DataFrame`.

In [74]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

array([ 0.5130905])

This person has a 51% chance of having a boy (according to the model).

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [75]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

The following are the only variables I found that have a statistically significant effect on pregnancy length.

In [76]:
import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()


AttributeError: 'OLSResults' object has no attribute 'SummarizeResults'

**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?

In [77]:
import regression
join = regression.JoinFemResp(live)

In [79]:
t=[]
for name in join.columns:
    try:
        if join[name].var()<1e-7:
            continue
        formula = 'totalwgt_lb ~ agepreg + ' + name
        model = smf.ols(formula,data=join)
        if model.nobs < len(join)/2:
            continue
        results = model.fit()
    except (ValueError,TypeError):
        continue
    t.append((results.rsquared,name))

In [82]:
t.sort(reverse=True)
for mse,name in t[:30]:
    print(name,mse)

totalwgt_lb 1.0
birthwgt_lb 0.949812730598
lbw1 0.300824078447
prglngth 0.130125194886
wksgest 0.123400413634
agecon 0.102031499282
mosgest 0.0271442746396
babysex 0.0185509252939
race_r 0.0161995035863
race 0.0161995035863
nbrnaliv 0.0160177527098
paydu 0.0140037955781
rmarout03 0.0134300664657
birthwgt_oz 0.0131024576157
anynurse 0.0125290225418
bfeedwks 0.0121936884045
totincr 0.0118700690312
marout03 0.0118078019944
marcon03 0.0117525993544
cebow 0.0114377709196
rmarout01 0.0114077371386
rmarout6 0.0113541384728
marout01 0.0112693572468
hisprace_r 0.011238349302
hisprace 0.011238349302
mar1diss 0.0109615635908
fmarcon5 0.0106049646843
rmarout02 0.0105469132066
marcon02 0.0104814017955
fmarout5 0.0104616913674


In [ ]:
# Solution goes here

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [83]:
formula= 'numbabes ~ agepreg + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.706263
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Sun, 19 Mar 2017   Pseudo R-squ.:                 0.02006
Time:                        15:30:41   Log-Likelihood:                -15158.
converged:                       True   LL-Null:                       -15469.
                                        LLR p-value:                7.374e-132
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        1.5208      0.038     39.687      0.000         1.446     1.596
C(race)[T.2]    -0.1588      0.015    -10.681      0.000        -0.188    -0.130
C(race)[T.3]    -0.1245      0.025     -5.066      0.000        -0.173    -0.076
agepreg          0.0114      0.001      9.117      0.000         0.009     0.014
totincr         -0.0137      0.002     -7.241      0.000        -0.017    -0.010
educat          -0.0471      0.003    -15.549      0.000        -0.053    -0.041
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [86]:
columns = ['agepreg', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35,1,14,16]], columns=columns)
results.predict(new)

array([ 2.64897185])

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [87]:
formula= 'rmarital ~ age_r + C(race) + educat + totincr'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

/home/willem/anaconda2/lib/python2.7/site-packages/statsmodels/discrete/discrete_model.py:130: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  endog_dummies = get_dummies(endog.icol(0))
/home/willem/anaconda2/lib/python2.7/site-packages/statsmodels/discrete/discrete_model.py:580: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  start_params = np.zeros((self.K * (self.J-1)))
/home/willem/anaconda2/lib/python2.7/site-packages/statsmodels/discrete/discrete_model.py:1840: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  params = params.reshape(self.K, -1, order='F')
/home/willem/anaconda2/lib/python2.7/site-packages/statsmodels/discrete/discrete_model.py:1756: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  params = params.reshape(self.K, -1, order='F')
/home/willem/anaconda2/l

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Sun, 19 Mar 2017   Pseudo R-squ.:                  0.1655
Time:                        15:36:59   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
                                        LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000         3.907     5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000        -1.095    -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000        -0.899    -0.368
age_r           -0.0570      0.006     -9.754      0.000        -0.068    -0.046
educat          -0.2051      0.019    -11.017      0.000        -0.242    -0.169
totincr         -0.1302      0.012    -11.298      0.000        -0.153    -0.108
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000        -6.338    -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062        -0.904     0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922        -0.623     0.689
age_r            0.1301      0.019      6.833      0.000         0.093     0.167
educat          -0.1010      0.048     -2.100      0.036        -0.195    -0.007
totincr         -0.3251      0.032    -10.182      0.000        -0.388    -0.263
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000        -3.239    -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001        -0.502    -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000        -1.101    -0.431
age_r            0.0641      0.007      9.817      0.000         0.051     0.077
educat           0.0673      0.017      4.036      0.000         0.035     0.100
totincr         -0.2270      0.012    -19.588      0.000        -0.250    -0.204
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017         0.147     1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000        -1.252    -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000        -0.870    -0.259
age_r            0.0063      0.007      0.872      0.383        -0.008     0.020
educat          -0.0136      0.021     -0.639      0.523        -0.056     0.028
totincr         -0.2923      0.015    -20.149      0.000        -0.321    -0.264
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000         5.697     6.777
C

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [88]:
import pandas
columns = ['age_r','race','educat','totincr']
new = pandas.DataFrame([[25, 2, 12, 11]], columns=columns)
results.predict(new)
#married, cohabitating, widowed, divorced, separated, or never married

array([[ 0.74838383,  0.12547442,  0.00110294,  0.03529465,  0.02381305,
         0.06593111]])